# Workflow s ľudským zásahom pomocou Microsoft Agent Framework

## 🎯 Ciele učenia

V tomto notebooku sa naučíte, ako implementovať **workflow s ľudským zásahom** pomocou `RequestInfoExecutor` v Microsoft Agent Framework. Tento výkonný vzor umožňuje pozastaviť AI workflowy na získanie vstupu od človeka, čím sa agenti stávajú interaktívnymi a ľudia získavajú kontrolu nad kritickými rozhodnutiami.

## 🔄 Čo je workflow s ľudským zásahom?

**Workflow s ľudským zásahom (HITL)** je návrhový vzor, pri ktorom AI agenti pozastavia vykonávanie, aby požiadali o vstup od človeka pred pokračovaním. To je nevyhnutné pre:

- ✅ **Kritické rozhodnutia** - Získajte schválenie od človeka pred vykonaním dôležitých akcií
- ✅ **Nejasné situácie** - Umožnite ľuďom objasniť, keď si AI nie je istá
- ✅ **Preferencie používateľa** - Požiadajte používateľov, aby si vybrali medzi viacerými možnosťami
- ✅ **Súlad a bezpečnosť** - Zabezpečte dohľad človeka pri regulovaných operáciách
- ✅ **Interaktívne zážitky** - Vytvorte konverzačných agentov, ktorí reagujú na vstupy používateľov

## 🏗️ Ako to funguje v Microsoft Agent Framework

Framework poskytuje tri kľúčové komponenty pre HITL:

1. **`RequestInfoExecutor`** - Špeciálny executor, ktorý pozastaví workflow a vydá `RequestInfoEvent`
2. **`RequestInfoMessage`** - Základná trieda pre typizované požiadavky odosielané ľuďom
3. **`RequestResponse`** - Spája odpovede od ľudí s pôvodnými požiadavkami pomocou `request_id`

**Vzor workflowu:**
```
Agent detects need for input
    ↓
Sends message to RequestInfoExecutor
    ↓
Workflow pauses & emits RequestInfoEvent
    ↓
Application collects human input (console, UI, etc.)
    ↓
Application sends RequestResponse via send_responses_streaming()
    ↓
Workflow resumes with human input
```

## 🏨 Náš príklad: Rezervácia hotela s potvrdením od používateľa

Rozšírime podmienený workflow pridaním ľudského potvrdenia **predtým**, než navrhneme alternatívne destinácie:

1. Používateľ požiada o destináciu (napr. "Paríž")
2. `availability_agent` skontroluje, či sú dostupné izby
3. **Ak nie sú izby** → `confirmation_agent` sa opýta "Chcete vidieť alternatívy?"
4. Workflow sa **pozastaví** pomocou `RequestInfoExecutor`
5. **Človek odpovie** "áno" alebo "nie" cez konzolový vstup
6. `decision_manager` rozhodne na základe odpovede:
   - **Áno** → Zobraziť alternatívne destinácie
   - **Nie** → Zrušiť požiadavku na rezerváciu
7. Zobraziť konečný výsledok

Tento príklad ukazuje, ako dať používateľom kontrolu nad návrhmi agenta!

---

Začnime! 🚀


## Krok 1: Importovanie potrebných knižníc

Importujeme štandardné komponenty Agent Frameworku plus **špecifické triedy pre zapojenie človeka do procesu**:
- `RequestInfoExecutor` - Executor, ktorý pozastaví pracovný postup pre vstup od človeka
- `RequestInfoEvent` - Udalosť, ktorá sa vyvolá, keď je požadovaný vstup od človeka
- `RequestInfoMessage` - Základná trieda pre typizované požiadavky na vstup
- `RequestResponse` - Koreluje odpovede od človeka s požiadavkami
- `WorkflowOutputEvent` - Udalosť na detekciu výstupov pracovného postupu


In [21]:
import asyncio
import json
import os
from dataclasses import dataclass
from typing import Annotated, Any, Never

from agent_framework import (
    AgentExecutor,
    AgentExecutorRequest,
    AgentExecutorResponse,
    ChatMessage,
    Executor,
    RequestInfoEvent,          # NEW: Event when human input is requested
    RequestInfoExecutor,       # NEW: Executor that gathers human input
    RequestInfoMessage,        # NEW: Base class for request payloads
    RequestResponse,           # NEW: Correlates response with request
    Role,
    WorkflowBuilder,
    WorkflowContext,
    WorkflowOutputEvent,       # NEW: Event for workflow outputs
    WorkflowRunState,          # NEW: Enum of workflow run states
    WorkflowStatusEvent,       # NEW: Event for run state changes
    ai_function,
    executor,
    handler,                   # NEW: Decorator for executor methods
)

# 🤖 GitHub Models or OpenAI client integration
from agent_framework.openai import OpenAIChatClient
from dotenv import load_dotenv
from IPython.display import HTML, display
from pydantic import BaseModel

print("✅ All imports successful!")
print("🔄 Human-in-the-loop components loaded: RequestInfoExecutor, RequestInfoEvent, RequestResponse")

✅ All imports successful!
🔄 Human-in-the-loop components loaded: RequestInfoExecutor, RequestInfoEvent, RequestResponse


## Krok 2: Definovanie Pydantic modelov pre štruktúrované výstupy

Tieto modely definujú **schému**, ktorú budú agenti vracať. Zachovávame všetky modely z podmieneného pracovného toku a pridávame:

**Nové pre zapojenie človeka do procesu:**
- `HumanFeedbackRequest` - Podtrieda `RequestInfoMessage`, ktorá definuje požiadavku odoslanú ľuďom
  - Obsahuje `prompt` (otázku na položenie) a `destination` (kontext o nedostupnom meste)


In [22]:
# Existing models from conditional workflow
class BookingCheckResult(BaseModel):
    """Result from checking hotel availability at a destination."""
    destination: str
    has_availability: bool
    message: str


class AlternativeResult(BaseModel):
    """Suggested alternative destination when no rooms available."""
    alternative_destination: str
    reason: str


class BookingConfirmation(BaseModel):
    """Booking suggestion when rooms are available."""
    destination: str
    action: str
    message: str


# NEW: Pydantic model for agent's response format
class ConfirmationQuestion(BaseModel):
    """
    Pydantic model used by confirmation_agent's response_format.
    This is what the agent will output as JSON.
    """
    question: str  # The question to ask the user
    destination: str  # The unavailable destination for context


# NEW: Dataclass for RequestInfoExecutor
@dataclass
class HumanFeedbackRequest(RequestInfoMessage):
    """
    Request sent to RequestInfoExecutor asking if user wants alternatives.
    
    MUST be a dataclass subclassing RequestInfoMessage for type compatibility.
    This is what gets sent to the RequestInfoExecutor.
    """
    prompt: str = ""  # The question to ask the user
    destination: str = ""  # The unavailable destination for context


print("✅ Pydantic models defined:")
print("   - BookingCheckResult (availability check)")
print("   - AlternativeResult (alternative suggestion)")
print("   - BookingConfirmation (booking confirmation)")
print("   - ConfirmationQuestion (agent response format) 🆕")
print("   - HumanFeedbackRequest (RequestInfoMessage for HITL) 🆕")

✅ Pydantic models defined:
   - BookingCheckResult (availability check)
   - AlternativeResult (alternative suggestion)
   - BookingConfirmation (booking confirmation)
   - ConfirmationQuestion (agent response format) 🆕
   - HumanFeedbackRequest (RequestInfoMessage for HITL) 🆕


## Krok 3: Vytvorte nástroj na rezerváciu hotelov

Rovnaký nástroj z podmieneného pracovného postupu - kontroluje, či sú v cieľovej destinácii dostupné izby.


In [23]:
@ai_function(description="Check hotel room availability for a destination city")
def hotel_booking(destination: Annotated[str, "The destination city to check for hotel rooms"]) -> str:
    """
    Simulates checking hotel room availability.
    
    Returns JSON string with availability status.
    """
    display(
        HTML(f"""
        <div style='padding: 15px; background: #e3f2fd; border-left: 4px solid #2196f3; border-radius: 4px; margin: 10px 0;'>
            <strong>🔍 Tool Invoked:</strong> hotel_booking("{destination}")
        </div>
    """)
    )

    # Simulate availability check
    cities_with_rooms = ["stockholm", "seattle", "tokyo", "london", "amsterdam"]
    has_rooms = destination.lower() in cities_with_rooms

    result = {"has_availability": has_rooms, "destination": destination}

    return json.dumps(result)


print("✅ hotel_booking tool created with @ai_function decorator")

✅ hotel_booking tool created with @ai_function decorator


## Krok 4: Definovanie podmienkových funkcií pre smerovanie

Potrebujeme **štyri podmienkové funkcie** pre náš pracovný postup s ľudským zásahom:

**Z podmieneného pracovného postupu:**
1. `has_availability_condition` - Smeruje, keď sú hotely DOSTUPNÉ
2. `no_availability_condition` - Smeruje, keď hotely NIE SÚ DOSTUPNÉ

**Nové pre pracovný postup s ľudským zásahom:**
3. `user_wants_alternatives_condition` - Smeruje, keď používateľ odpovie "áno" na alternatívy
4. `user_declines_alternatives_condition` - Smeruje, keď používateľ odpovie "nie" na alternatívy


In [24]:
# Existing condition functions from conditional workflow
def has_availability_condition(message: Any) -> bool:
    """Condition for routing when hotels ARE available."""
    if not isinstance(message, AgentExecutorResponse):
        return True

    try:
        result = BookingCheckResult.model_validate_json(message.agent_run_response.text)
        display(
            HTML(f"""
            <div style='padding: 12px; background: #c8e6c9; border-left: 4px solid #4caf50; border-radius: 4px; margin: 10px 0;'>
                <strong>✅ Condition Check:</strong> has_availability = <strong>{result.has_availability}</strong> for {result.destination}
            </div>
        """)
        )
        return result.has_availability
    except Exception as e:
        display(HTML(f"""<div style='padding: 12px; background: #ffcdd2; border-left: 4px solid #f44336; border-radius: 4px; margin: 10px 0;'><strong>⚠️  Error:</strong> {str(e)}</div>"""))
        return False


def no_availability_condition(message: Any) -> bool:
    """Condition for routing when hotels are NOT available."""
    if not isinstance(message, AgentExecutorResponse):
        return False

    try:
        result = BookingCheckResult.model_validate_json(message.agent_run_response.text)
        display(
            HTML(f"""
            <div style='padding: 12px; background: #ffecb3; border-left: 4px solid #ff9800; border-radius: 4px; margin: 10px 0;'>
                <strong>❌ Condition Check:</strong> no_availability for {result.destination}
            </div>
        """)
        )
        return not result.has_availability
    except Exception as e:
        return False


# NEW: Condition functions for human-in-the-loop routing
def user_wants_alternatives_condition(message: Any) -> bool:
    """
    Condition for routing when user WANTS to see alternatives.
    
    Checks the AgentExecutorRequest sent by decision_manager.
    """
    # Check if it's an AgentExecutorRequest (what decision_manager sends)
    if isinstance(message, AgentExecutorRequest):
        # Check the message text to determine user's choice
        if message.messages and len(message.messages) > 0:
            msg_text = message.messages[0].text.lower()
            wants_alternatives = "wants to see alternative" in msg_text or "want to see alternative" in msg_text
            
            display(
                HTML(f"""
                <div style='padding: 12px; background: #e1f5fe; border-left: 4px solid #0288d1; border-radius: 4px; margin: 10px 0;'>
                    <strong>🔍 User Decision:</strong> User wants alternatives = <strong>{wants_alternatives}</strong>
                </div>
            """)
            )
            
            return wants_alternatives
    
    return False
def user_declines_alternatives_condition(message: Any) -> bool:
    """
    Condition for routing when user DECLINES alternatives.
    
    Checks the AgentExecutorRequest sent by decision_manager.
    """
    # Check if it's an AgentExecutorRequest (what decision_manager sends)
    if isinstance(message, AgentExecutorRequest):
        # Check the message text to determine user's choice
        if message.messages and len(message.messages) > 0:
            msg_text = message.messages[0].text.lower()
            declined = "declined" in msg_text or "has declined" in msg_text
            
            display(
                HTML(f"""
                <div style='padding: 12px; background: #fce4ec; border-left: 4px solid #c2185b; border-radius: 4px; margin: 10px 0;'>
                    <strong>🚫 User Decision:</strong> User declined alternatives = <strong>{declined}</strong>
                </div>
            """)
            )
            
            return declined
    
    return False
print("✅ Condition functions defined:")
print("   - has_availability_condition (routes when rooms exist)")
print("   - no_availability_condition (routes when no rooms)")
print("   - user_wants_alternatives_condition (routes when user says yes) 🆕")
print("   - user_declines_alternatives_condition (routes when user says no) 🆕")

✅ Condition functions defined:
   - has_availability_condition (routes when rooms exist)
   - no_availability_condition (routes when no rooms)
   - user_wants_alternatives_condition (routes when user says yes) 🆕
   - user_declines_alternatives_condition (routes when user says no) 🆕


## Krok 5: Vytvorenie Decision Manager Executor

Toto je **jadro vzoru "človek v slučke"**! `DecisionManager` je vlastný `Executor`, ktorý:

1. **Prijíma spätnú väzbu od človeka** prostredníctvom objektov `RequestResponse`
2. **Spracováva rozhodnutie používateľa** (áno/nie)
3. **Riadi pracovný tok** zasielaním správ vhodným agentom

Kľúčové vlastnosti:
- Používa dekorátor `@handler` na sprístupnenie metód ako krokov pracovného toku
- Prijíma `RequestResponse[HumanFeedbackRequest, str]`, ktorý obsahuje pôvodnú požiadavku aj odpoveď používateľa
- Vytvára jednoduché správy "áno" alebo "nie", ktoré spúšťajú naše podmienkové funkcie


In [25]:
class DecisionManager(Executor):
    """
    Coordinates workflow routing based on human feedback.
    
    This executor receives RequestResponse objects from the RequestInfoExecutor
    and makes routing decisions by sending simple messages that trigger
    condition functions.
    """

    def __init__(self, id: str | None = None):
        super().__init__(id=id or "decision_manager")

    @handler
    async def on_human_feedback(
        self,
        feedback: RequestResponse[HumanFeedbackRequest, str],
        ctx: WorkflowContext[AgentExecutorRequest],
    ) -> None:
        """
        Process human feedback and let the workflow route based on conditions.
        
        The RequestResponse contains:
        - feedback.data: The user's string reply (e.g., "yes" or "no")
        - feedback.original_request: The HumanFeedbackRequest with context
        
        This handler just displays feedback and passes the RequestResponse through.
        The routing is done by condition functions on the edges.
        """
        user_reply = (feedback.data or "").strip().lower()
        destination = getattr(feedback.original_request, "destination", "unknown")

        display(
            HTML(f"""
            <div style='padding: 15px; background: #f3e5f5; border-left: 4px solid #9c27b0; border-radius: 4px; margin: 10px 0;'>
                <strong>🎯 Decision Manager:</strong> Processing user reply: <strong>"{user_reply}"</strong> for {destination}
            </div>
        """)
        )

        if user_reply == "yes":
            display(
                HTML("""
                <div style='padding: 12px; background: #c8e6c9; border-left: 4px solid #4caf50; border-radius: 4px; margin: 10px 0;'>
                    <strong>➡️  Routing:</strong> User wants alternatives → Will route to alternative_agent
                </div>
            """)
            )
            # Create and send a message for the alternative_agent
            user_msg = ChatMessage(
                Role.USER,
                text=f"The user wants to see alternative destinations near {destination}. Please suggest one.",
            )
            await ctx.send_message(AgentExecutorRequest(messages=[user_msg], should_respond=True))
        
        elif user_reply == "no":
            display(
                HTML("""
                <div style='padding: 12px; background: #ffcdd2; border-left: 4px solid #f44336; border-radius: 4px; margin: 10px 0;'>
                    <strong>🚫 Routing:</strong> User declined alternatives → Will route to cancellation_agent
                </div>
            """)
            )
            # Create and send a message for the cancellation_agent
            user_msg = ChatMessage(
                Role.USER,
                text="The user has declined to see alternatives. Please acknowledge their decision.",
            )
            await ctx.send_message(AgentExecutorRequest(messages=[user_msg], should_respond=True))
        
        else:
            # Handle unexpected input - treat as decline
            display(
                HTML(f"""
                <div style='padding: 12px; background: #fff3e0; border-left: 4px solid #ff9800; border-radius: 4px; margin: 10px 0;'>
                    <strong>⚠️  Warning:</strong> Unexpected input "{user_reply}" - treating as decline
                </div>
            """)
            )
            user_msg = ChatMessage(
                Role.USER,
                text="The user has declined to see alternatives. Please acknowledge their decision.",
            )
            await ctx.send_message(AgentExecutorRequest(messages=[user_msg], should_respond=True))


print("✅ DecisionManager executor created with @handler method for human feedback")

✅ DecisionManager executor created with @handler method for human feedback


## Krok 6: Vytvorenie vlastného vykonávateľa zobrazenia

Rovnaký vykonávateľ zobrazenia z podmieneného pracovného toku - poskytuje konečné výsledky ako výstup pracovného toku.


In [26]:
@executor(id="prepare_human_request")
async def prepare_human_request(
    response: AgentExecutorResponse, 
    ctx: WorkflowContext[HumanFeedbackRequest]
) -> None:
    """
    Transform agent response into HumanFeedbackRequest for RequestInfoExecutor.
    
    This executor bridges the type gap between:
    - confirmation_agent outputs AgentExecutorResponse with ConfirmationQuestion JSON
    - request_info_executor expects HumanFeedbackRequest (RequestInfoMessage dataclass)
    """
    display(
        HTML("""
        <div style='padding: 12px; background: #e1f5fe; border-left: 4px solid #0288d1; border-radius: 4px; margin: 10px 0;'>
            <strong>🔄 Transform:</strong> Converting ConfirmationQuestion to HumanFeedbackRequest
        </div>
    """)
    )
    
    # Parse the agent's Pydantic output (ConfirmationQuestion)
    confirmation = ConfirmationQuestion.model_validate_json(response.agent_run_response.text)
    
    # Convert to HumanFeedbackRequest dataclass for RequestInfoExecutor
    feedback_request = HumanFeedbackRequest(
        prompt=confirmation.question,
        destination=confirmation.destination
    )
    
    # Send the properly typed RequestInfoMessage to the RequestInfoExecutor
    await ctx.send_message(feedback_request)


@executor(id="display_result")
async def display_result(response: AgentExecutorResponse, ctx: WorkflowContext[Never, str]) -> None:
    """
    Display the final result as workflow output.
    
    This executor receives the final agent response and yields it as the workflow output.
    """
    display(
        HTML("""
        <div style='padding: 15px; background: #f3e5f5; border-left: 4px solid #9c27b0; border-radius: 4px; margin: 10px 0;'>
            <strong>📤 Display Executor:</strong> Yielding workflow output
        </div>
    """)
    )

    await ctx.yield_output(response.agent_run_response.text)


print("✅ prepare_human_request executor created with @executor decorator")
print("✅ display_result executor created with @executor decorator")

✅ prepare_human_request executor created with @executor decorator
✅ display_result executor created with @executor decorator


## Krok 7: Načítanie environmentálnych premenných

Nakonfigurujte klienta LLM (GitHub Models, Azure OpenAI alebo OpenAI).


In [27]:
# Load environment variables
load_dotenv()

# Check for GitHub Models or OpenAI
chat_client = OpenAIChatClient(
    base_url=os.environ.get("GITHUB_ENDPOINT"), 
    api_key=os.environ.get("GITHUB_TOKEN"), 
    model_id="gpt-4o"
)

print("✅ Chat client configured with GitHub Models")

✅ Chat client configured with GitHub Models


## Krok 8: Vytvorenie AI agentov a vykonávateľov

Vytvárame **šesť komponentov pracovného toku**:

**Agenti (zabalení v AgentExecutor):**
1. **availability_agent** - Kontroluje dostupnosť hotelov pomocou nástroja
2. **confirmation_agent** - 🆕 Pripravuje žiadosť o potvrdenie od človeka
3. **alternative_agent** - Navrhuje alternatívne mestá (keď používateľ povie áno)
4. **booking_agent** - Podporuje rezerváciu (keď sú dostupné izby)
5. **cancellation_agent** - 🆕 Spracováva správu o zrušení (keď používateľ povie nie)

**Špeciálni vykonávateľia:**
6. **request_info_executor** - 🆕 `RequestInfoExecutor`, ktorý pozastaví pracovný tok pre vstup od človeka
7. **decision_manager** - 🆕 Vlastný vykonávateľ, ktorý smeruje na základe odpovede od človeka (už definovaný vyššie)


In [28]:
# Agent 1: Check availability with tool (same as conditional workflow)
availability_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a hotel booking assistant that checks room availability. "
            "Use the hotel_booking tool to check if rooms are available at the destination. "
            "Return JSON with fields: destination (string), has_availability (bool), and message (string). "
            "The message should summarize the availability status."
        ),
        tools=[hotel_booking],
        response_format=BookingCheckResult,
    ),
    id="availability_agent",
)

# Agent 2: NEW - Prepare human confirmation request
confirmation_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a helpful assistant. The user's requested destination has no available hotel rooms. "
            "Create a polite message asking if they would like to see alternative destinations nearby. "
            "Return a JSON with: destination (the unavailable city), and question (a friendly yes/no question). "
            "Keep the question concise and friendly."
        ),
        response_format=ConfirmationQuestion,  # Use Pydantic model for agent output
    ),
    id="confirmation_agent",
)

# Agent 3: Suggest alternative (when user says yes)
alternative_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a helpful travel assistant. When a user cannot find hotels in their requested city, "
            "suggest an alternative nearby city that has availability. "
            "Return JSON with fields: alternative_destination (string) and reason (string). "
            "Make your suggestion sound appealing and helpful."
        ),
        response_format=AlternativeResult,
    ),
    id="alternative_agent",
)

# Agent 4: Suggest booking (when rooms available)
booking_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a booking assistant. The user has found available hotel rooms. "
            "Encourage them to book by highlighting the destination's appeal. "
            "Return JSON with fields: destination (string), action (string), and message (string). "
            "The action should be 'book_now' and message should be encouraging."
        ),
        response_format=BookingConfirmation,
    ),
    id="booking_agent",
)

# Agent 5: NEW - Handle cancellation when user declines alternatives
class CancellationMessage(BaseModel):
    """Message when user declines alternatives."""
    status: str
    message: str

cancellation_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a helpful assistant. The user has declined to see alternative hotel destinations. "
            "Create a polite cancellation message. "
            "Return JSON with: status (should be 'cancelled'), and message (a friendly acknowledgment). "
            "Keep the message brief and understanding."
        ),
        response_format=CancellationMessage,
    ),
    id="cancellation_agent",
)

# NEW: RequestInfoExecutor - pauses workflow to gather human input
request_info_executor = RequestInfoExecutor(id="request_info")

# NEW: DecisionManager instance - routes based on human feedback
decision_manager = DecisionManager(id="decision_manager")

display(
    HTML("""
    <div style='padding: 15px; background: #e3f2fd; border-left: 4px solid #2196f3; border-radius: 4px; margin: 10px 0;'>
        <strong>✅ Created Workflow Components:</strong>
        <ul style='margin: 10px 0 0 0;'>
            <li><strong>availability_agent</strong> - Checks availability with hotel_booking tool</li>
            <li><strong>confirmation_agent</strong> 🆕 - Prepares human confirmation request</li>
            <li><strong>alternative_agent</strong> - Suggests alternative cities</li>
            <li><strong>booking_agent</strong> - Encourages booking</li>
            <li><strong>cancellation_agent</strong> 🆕 - Handles user declining alternatives</li>
            <li><strong>request_info_executor</strong> 🆕 - Pauses workflow for human input</li>
            <li><strong>decision_manager</strong> 🆕 - Routes based on human response</li>
        </ul>
    </div>
""")
)

## Krok 9: Vytvorenie pracovného toku s ľudským zásahom

Teraz zostavíme graf pracovného toku s **podmieneným smerovaním**, vrátane cesty s ľudským zásahom:

**Štruktúra pracovného toku:**
```
availability_agent (START)
        ↓
   Evaluate conditions
        ↙                    ↘
[no_availability]        [has_availability]
        ↓                        ↓
confirmation_agent          booking_agent
        ↓                        ↓
prepare_human_request      display_result
        ↓
request_info_executor (PAUSE)
        ↓
decision_manager
   ↙         ↘
[yes]        [no]
   ↓           ↓
alternative  cancellation
   ↓           ↓
display_result
```

**Kľúčové prepojenia:**
- `availability_agent → confirmation_agent` (keď nie sú dostupné izby)
- `confirmation_agent → prepare_human_request` (transformácia typu)
- `prepare_human_request → request_info_executor` (pozastavenie pre človeka)
- `request_info_executor → decision_manager` (vždy - poskytuje RequestResponse)
- `decision_manager → alternative_agent` (keď používateľ povie "áno")
- `decision_manager → cancellation_agent` (keď používateľ povie "nie")
- `availability_agent → booking_agent` (keď sú dostupné izby)
- Všetky cesty končia na `display_result`


In [29]:
# Build the workflow with human-in-the-loop routing
workflow = (
    WorkflowBuilder()
    .set_start_executor(availability_agent)
    
    # NO AVAILABILITY PATH (with human-in-the-loop)
    .add_edge(availability_agent, confirmation_agent, condition=no_availability_condition)
    .add_edge(confirmation_agent, prepare_human_request)  # Transform to HumanFeedbackRequest
    .add_edge(prepare_human_request, request_info_executor)  # Send to RequestInfoExecutor
    .add_edge(request_info_executor, decision_manager)    # Always goes to decision manager
    
    # Decision manager routes based on user response
    .add_edge(decision_manager, alternative_agent, condition=user_wants_alternatives_condition)
    .add_edge(decision_manager, cancellation_agent, condition=user_declines_alternatives_condition)
    .add_edge(alternative_agent, display_result)
    .add_edge(cancellation_agent, display_result)
    
    # HAS AVAILABILITY PATH (no human input needed)
    .add_edge(availability_agent, booking_agent, condition=has_availability_condition)
    .add_edge(booking_agent, display_result)
    
    .build()
)

display(
    HTML("""
    <div style='padding: 20px; background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); color: white; border-radius: 8px; margin: 10px 0;'>
        <h3 style='margin: 0 0 15px 0;'>✅ Workflow Built Successfully!</h3>
        <p style='margin: 0; line-height: 1.6;'>
            <strong>Human-in-the-Loop Routing:</strong><br>
            • If <strong>NO availability</strong> → confirmation_agent → prepare_human_request → request_info_executor → <strong>PAUSE FOR HUMAN</strong> → decision_manager<br>
            &nbsp;&nbsp;• If user says <strong>YES</strong> → alternative_agent → display_result<br>
            &nbsp;&nbsp;• If user says <strong>NO</strong> → cancellation_agent → display_result<br>
            • If <strong>availability</strong> → booking_agent → display_result (no human input needed)
        </p>
    </div>
""")
)

## Krok 10: Spustite testovací prípad 1 - Mesto BEZ dostupnosti (Paríž s ľudským potvrdením)

Tento test demonštruje **kompletný cyklus s ľudským zásahom**:

1. Požiadavka na hotel v Paríži
2. availability_agent kontroluje → Žiadne voľné izby
3. confirmation_agent vytvorí otázku pre človeka
4. request_info_executor **pozastaví pracovný postup** a vydá `RequestInfoEvent`
5. **Aplikácia detekuje udalosť a vyzve používateľa v konzole**
6. Používateľ napíše "áno" alebo "nie"
7. Aplikácia odošle odpoveď cez `send_responses_streaming()`
8. decision_manager rozhoduje na základe odpovede
9. Zobrazený konečný výsledok

**Kľúčový vzor:**
- Použite `workflow.run_stream()` pre prvú iteráciu
- Použite `workflow.send_responses_streaming(pending_responses)` pre nasledujúce iterácie
- Počúvajte na `RequestInfoEvent`, aby ste zistili, kedy je potrebný ľudský vstup
- Počúvajte na `WorkflowOutputEvent`, aby ste zachytili konečné výsledky


In [ ]:
display(
    HTML("""
    <div style='padding: 20px; background: #fff3e0; border-left: 4px solid #ff9800; border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #e65100;'>🧪 TEST CASE 1: Paris (No Availability - Human-in-the-Loop)</h3>
        <p style='margin: 0;'>Expected workflow path: availability_agent → confirmation_agent → request_info_executor → <strong>PAUSE</strong> → decision_manager → (depends on user input)</p>
    </div>
""")
)

# Create request for Paris
request_paris = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Paris")], 
    should_respond=True
)

# Human-in-the-loop execution pattern
pending_responses: dict[str, str] | None = None
completed = False
workflow_output: str | None = None

print("\n🔄 Starting human-in-the-loop workflow...")
print("=" * 60)

while not completed:
    # First iteration uses run_stream with the request
    # Subsequent iterations use send_responses_streaming with collected human responses
    if pending_responses:
        print(f"\n📤 Sending human responses: {pending_responses}")
        stream = workflow.send_responses_streaming(pending_responses)
        pending_responses = None  # Clear immediately after sending
    else:
        print(f"\n🚀 Starting workflow with request: 'I want to book a hotel in Paris'")
        stream = workflow.run_stream(request_paris)
    
    # Collect all events from this iteration
    events = [event async for event in stream]
    
    # Process events
    requests: list[tuple[str, str]] = []  # (request_id, prompt)
    
    for event in events:
        # Check for human input requests
        if isinstance(event, RequestInfoEvent) and isinstance(event.data, HumanFeedbackRequest):
            print(f"\n⏸️  WORKFLOW PAUSED - Human input requested!")
            print(f"   Request ID: {event.request_id}")
            print(f"   Destination: {event.data.destination}")
            requests.append((event.request_id, event.data.prompt))
        
        # Check for workflow outputs
        elif isinstance(event, WorkflowOutputEvent):
            workflow_output = str(event.data)
            completed = True
            print(f"\n✅ Workflow completed with output!")
    
    # If we have human requests, prompt the user
    if requests and not completed:
        responses: dict[str, str] = {}
        for req_id, prompt in requests:
            print(f"\n{'='*60}")
            print(f"💬 QUESTION FOR YOU:")
            print(f"   {prompt}")
            print(f"{'='*60}")
            
            # Get user input (in notebook, this will pause execution)
            answer = input("👉 Enter 'yes' or 'no': ").strip().lower()
            
            print(f"\n📝 You answered: {answer}")
            responses[req_id] = answer
        
        pending_responses = responses

print(f"\n{'='*60}")
print(f"🏆 FINAL WORKFLOW OUTPUT:")
print(f"{'='*60}")

# Display final result
if workflow_output:
    # Try to parse as JSON for pretty display
    try:
        result_data = json.loads(workflow_output)
        if "alternative_destination" in result_data:
            result_obj = AlternativeResult.model_validate_json(workflow_output)
            display(
                HTML(f"""
                <div style='padding: 25px; background: linear-gradient(135deg, #FFD700 0%, #FFA500 100%); border-radius: 12px; box-shadow: 0 4px 12px rgba(255,165,0,0.3); margin: 20px 0;'>
                    <h3 style='margin: 0 0 15px 0; color: #333;'>🏆 WORKFLOW RESULT</h3>
                    <div style='background: white; padding: 20px; border-radius: 8px;'>
                        <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ❌ No rooms in Paris</p>
                        <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>User Decision:</strong> ✅ Accepted alternatives</p>
                        <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Alternative Suggestion:</strong> 🏨 {result_obj.alternative_destination}</p>
                        <p style='margin: 0; font-size: 14px; color: #666;'><strong>Reason:</strong> {result_obj.reason}</p>
                    </div>
                </div>
            """)
            )
        else:
            # User declined
            display(
                HTML(f"""
                <div style='padding: 25px; background: linear-gradient(135deg, #f44336 0%, #e91e63 100%); color: white; border-radius: 12px; box-shadow: 0 4px 12px rgba(244,67,54,0.3); margin: 20px 0;'>
                    <h3 style='margin: 0 0 15px 0;'>🏆 WORKFLOW RESULT</h3>
                    <div style='background: white; color: #333; padding: 20px; border-radius: 8px;'>
                        <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ❌ No rooms in Paris</p>
                        <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>User Decision:</strong> 🚫 Declined alternatives</p>
                        <p style='margin: 0; font-size: 14px; color: #666;'><strong>Result:</strong> Booking request cancelled</p>
                    </div>
                </div>
            """)
            )
    except:
        print(workflow_output)


🔄 Starting human-in-the-loop workflow...

🚀 Starting workflow with request: 'I want to book a hotel in Paris'



⏸️  WORKFLOW PAUSED - Human input requested!
   Request ID: 032c8fce-b9d1-400e-ba8d-afd2248e2926
   Destination: Paris

💬 QUESTION FOR YOU:
   Unfortunately, there are no rooms available in Paris. Would you like to explore nearby alternative destinations?

📝 You answered: yes

📤 Sending human responses: {'032c8fce-b9d1-400e-ba8d-afd2248e2926': 'yes'}

🚀 Starting workflow with request: 'I want to book a hotel in Paris'

📝 You answered: yes

📤 Sending human responses: {'032c8fce-b9d1-400e-ba8d-afd2248e2926': 'yes'}

🚀 Starting workflow with request: 'I want to book a hotel in Paris'



⏸️  WORKFLOW PAUSED - Human input requested!
   Request ID: cf48dad0-ee5e-4f60-8806-341a7a292bd4
   Destination: Paris

💬 QUESTION FOR YOU:
   I'm sorry to inform you that there are no available hotel rooms in Paris. Would you like me to suggest nearby alternative destinations?

📝 You answered: 

📤 Sending human responses: {'cf48dad0-ee5e-4f60-8806-341a7a292bd4': ''}

🚀 Starting workflow with request: 'I want to book a hotel in Paris'

📝 You answered: 

📤 Sending human responses: {'cf48dad0-ee5e-4f60-8806-341a7a292bd4': ''}

🚀 Starting workflow with request: 'I want to book a hotel in Paris'


## Krok 11: Spustite testovací prípad 2 - Mesto S dostupnosťou (Štokholm - Nie je potrebný žiadny ľudský vstup)

Tento test demonštruje **priamu cestu**, keď sú izby dostupné:

1. Požiadajte o hotel v Štokholme
2. availability_agent kontroluje → Izby dostupné ✅
3. booking_agent navrhuje rezerváciu
4. display_result zobrazí potvrdenie
5. **Nie je potrebný žiadny ľudský vstup!**

Pracovný postup úplne obchádza cestu s ľudským zásahom, keď sú izby dostupné.


In [ ]:
display(
    HTML("""
    <div style='padding: 20px; background: #e8f5e9; border-left: 4px solid #4caf50; border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #1b5e20;'>🧪 TEST CASE 2: Stockholm (Has Availability - No Human Input)</h3>
        <p style='margin: 0;'>Expected workflow path: availability_agent → booking_agent → display_result (direct, no pause)</p>
    </div>
""")
)

# Create request for Stockholm
request_stockholm = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Stockholm")], 
    should_respond=True
)

# Run the workflow (should complete without human input)
events_stockholm = await workflow.run(request_stockholm)
outputs_stockholm = events_stockholm.get_outputs()

# Display results
if outputs_stockholm:
    result_stockholm = BookingConfirmation.model_validate_json(outputs_stockholm[0])

    display(
        HTML(f"""
        <div style='padding: 25px; background: linear-gradient(135deg, #4caf50 0%, #8bc34a 100%); color: white; border-radius: 12px; box-shadow: 0 4px 12px rgba(76,175,80,0.3); margin: 20px 0;'>
            <h3 style='margin: 0 0 15px 0;'>🏆 WORKFLOW RESULT (Stockholm - No Human Input)</h3>
            <div style='background: white; color: #333; padding: 20px; border-radius: 8px;'>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ✅ Rooms Available!</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Destination:</strong> 🏨 {result_stockholm.destination}</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Action:</strong> {result_stockholm.action}</p>
                <p style='margin: 0 0 10px 0; font-size: 14px; color: #666;'><strong>Message:</strong> {result_stockholm.message}</p>
                <p style='margin: 10px 0 0 0; font-size: 12px; color: #999; font-style: italic;'>Note: No human input was requested because rooms were available!</p>
            </div>
        </div>
    """)
    )

## Hlavné poznatky a osvedčené postupy pre zapojenie človeka do procesu

### ✅ Čo ste sa naučili:

#### 1. **RequestInfoExecutor Pattern**
Vzorec zapojenia človeka do procesu v Microsoft Agent Framework využíva tri kľúčové komponenty:
- `RequestInfoExecutor` - Pozastaví pracovný tok a generuje udalosti
- `RequestInfoMessage` - Základná trieda pre typizované požiadavky (rozšírte ju!)
- `RequestResponse` - Spája odpovede od človeka s pôvodnými požiadavkami

**Kľúčové pochopenie:**
- `RequestInfoExecutor` sám nezbiera vstupy - iba pozastaví pracovný tok
- Vaša aplikácia musí počúvať `RequestInfoEvent` a zbierať vstupy
- Musíte zavolať `send_responses_streaming()` s dict mapovaním `request_id` na odpoveď používateľa

#### 2. **Vzorec streamovanej exekúcie**
```python
# First iteration
stream = workflow.run_stream(initial_request)

# Subsequent iterations (after human input)
stream = workflow.send_responses_streaming(pending_responses)

# Always process events
events = [event async for event in stream]
```

#### 3. **Architektúra založená na udalostiach**
Počúvajte konkrétne udalosti na ovládanie pracovného toku:
- `RequestInfoEvent` - Potrebný je vstup od človeka (pracovný tok pozastavený)
- `WorkflowOutputEvent` - Konečný výsledok je dostupný (pracovný tok dokončený)
- `WorkflowStatusEvent` - Zmeny stavu (IN_PROGRESS, IDLE_WITH_PENDING_REQUESTS, atď.)

#### 4. **Vlastné exekútory s @handler**
`DecisionManager` demonštruje, ako vytvoriť exekútory, ktoré:
- Používajú dekorátor `@handler` na sprístupnenie metód ako krokov pracovného toku
- Prijímajú typizované správy (napr. `RequestResponse[HumanFeedbackRequest, str]`)
- Riadia pracovný tok posielaním správ iným exekútorom
- Pristupujú ku kontextu cez `WorkflowContext`

#### 5. **Podmienené smerovanie na základe rozhodnutí človeka**
Môžete vytvoriť funkcie podmienok, ktoré vyhodnocujú odpovede od človeka:
```python
def user_wants_alternatives_condition(message: Any) -> bool:
    response_text = message.agent_run_response.text.lower()
    return response_text == "yes"
```

### 🎯 Reálne aplikácie:

1. **Schvaľovacie pracovné toky**
   - Získajte schválenie od manažéra pred spracovaním výdavkových správ
   - Vyžadujte ľudskú kontrolu pred odoslaním automatizovaných e-mailov
   - Potvrďte transakcie s vysokou hodnotou pred ich vykonaním

2. **Moderovanie obsahu**
   - Označte sporný obsah na kontrolu človekom
   - Požiadajte moderátorov o konečné rozhodnutie v hraničných prípadoch
   - Eskalujte na človeka, keď je dôvera AI nízka

3. **Zákaznícky servis**
   - Nechajte AI automaticky riešiť rutinné otázky
   - Eskalujte zložité problémy na ľudských agentov
   - Opýtajte sa zákazníka, či chce hovoriť s človekom

4. **Spracovanie údajov**
   - Požiadajte ľudí, aby vyriešili nejednoznačné záznamy údajov
   - Potvrďte interpretácie AI nejasných dokumentov
   - Nechajte používateľov vybrať medzi viacerými platnými interpretáciami

5. **Systémy kritické pre bezpečnosť**
   - Vyžadujte potvrdenie od človeka pred nevratnými akciami
   - Získajte schválenie pred prístupom k citlivým údajom
   - Potvrďte rozhodnutia v regulovaných odvetviach (zdravotníctvo, financie)

6. **Interaktívni agenti**
   - Vytvorte konverzačné boty, ktoré kladú doplňujúce otázky
   - Vytvorte sprievodcov, ktorí vedú používateľov cez zložité procesy
   - Navrhnite agentov, ktorí spolupracujú s ľuďmi krok za krokom

### 🔄 Porovnanie: S vs Bez zapojenia človeka do procesu

| Funkcia | Podmienený pracovný tok | Pracovný tok so zapojením človeka |
|---------|-------------------------|----------------------------------|
| **Exekúcia** | Jednoduché `workflow.run()` | Slučka s `run_stream()` + `send_responses_streaming()` |
| **Vstup používateľa** | Žiadny (plne automatizované) | Interaktívne výzvy cez `input()` alebo UI |
| **Komponenty** | Agenti + Exekútory | + RequestInfoExecutor + DecisionManager |
| **Udalosti** | Iba AgentExecutorResponse | RequestInfoEvent, WorkflowOutputEvent, atď. |
| **Pozastavenie** | Bez pozastavenia | Pracovný tok sa pozastaví na RequestInfoExecutor |
| **Kontrola človekom** | Bez kontroly človekom | Ľudia robia kľúčové rozhodnutia |
| **Použitie** | Automatizácia | Spolupráca a dohľad |

### 🚀 Pokročilé vzorce:

#### Viacero bodov rozhodovania človeka
Môžete mať viacero uzlov `RequestInfoExecutor` v tom istom pracovnom toku:
```python
.add_edge(agent1, request_info_1)  # First human decision
.add_edge(decision_manager_1, agent2)
.add_edge(agent2, request_info_2)  # Second human decision
.add_edge(decision_manager_2, final_agent)
```

#### Riešenie časových limitov
Implementujte časové limity pre odpovede od človeka:
```python
import asyncio

try:
    answer = await asyncio.wait_for(
        asyncio.to_thread(input, "Enter yes/no: "),
        timeout=60.0
    )
except asyncio.TimeoutError:
    answer = "no"  # Default to safe option
```

#### Integrácia bohatého UI
Namiesto `input()` integrujte s webovým UI, Slack, Teams, atď.:
```python
if isinstance(event, RequestInfoEvent):
    # Send notification to user's preferred channel
    await slack_client.send_message(
        user_id=current_user,
        text=event.data.prompt,
        request_id=event.request_id
    )
```

#### Podmienené zapojenie človeka do procesu
Požiadajte o vstup od človeka iba v konkrétnych situáciách:
```python
def needs_human_approval_condition(message: Any) -> bool:
    # Only route to human if confidence is low or value is high
    if result.confidence < 0.7 or result.value > 10000:
        return True
    return False
```

### ⚠️ Osvedčené postupy:

1. **Vždy rozšírte RequestInfoMessage**
   - Poskytuje typovú bezpečnosť a validáciu
   - Umožňuje bohatý kontext pre vykreslenie UI
   - Ujasňuje zámer každého typu požiadavky

2. **Používajte popisné výzvy**
   - Zahrňte kontext o tom, na čo sa pýtate
   - Vysvetlite dôsledky každej voľby
   - Udržujte otázky jednoduché a jasné

3. **Riešte neočakávané vstupy**
   - Validujte odpovede používateľa
   - Poskytnite predvolené hodnoty pre neplatné vstupy
   - Poskytnite jasné chybové správy

4. **Sledujte ID požiadaviek**
   - Používajte koreláciu medzi request_id a odpoveďami
   - Nepokúšajte sa spravovať stav manuálne

5. **Navrhujte pre neblokovanie**
   - Neblokujte vlákna čakajúce na vstup
   - Používajte asynchrónne vzorce
   - Podporujte súbežné inštancie pracovného toku

### 📚 Súvisiace koncepty:

- **Agent Middleware** - Zachytávajte volania agentov (predchádzajúci notebook)
- **Správa stavu pracovného toku** - Ukladajte stav pracovného toku medzi spusteniami
- **Spolupráca viacerých agentov** - Kombinujte zapojenie človeka do procesu s tímami agentov
- **Architektúry založené na udalostiach** - Budujte reaktívne systémy s udalosťami

---

### 🎓 Gratulujeme!

Ovládli ste pracovné toky so zapojením človeka do procesu v Microsoft Agent Framework! Teraz viete:
- ✅ Pozastaviť pracovné toky na získanie vstupu od človeka
- ✅ Používať RequestInfoExecutor a RequestInfoMessage
- ✅ Riešiť streamovanú exekúciu pomocou udalostí
- ✅ Vytvárať vlastné exekútory s @handler
- ✅ Smerovať pracovné toky na základe rozhodnutí človeka
- ✅ Budovať interaktívnych AI agentov, ktorí spolupracujú s ľuďmi

**Toto je kľúčový vzorec pre budovanie dôveryhodných a kontrolovateľných AI systémov!** 🚀



---

**Zrieknutie sa zodpovednosti**:  
Tento dokument bol preložený pomocou služby AI prekladu [Co-op Translator](https://github.com/Azure/co-op-translator). Aj keď sa snažíme o presnosť, prosím, berte na vedomie, že automatizované preklady môžu obsahovať chyby alebo nepresnosti. Pôvodný dokument v jeho rodnom jazyku by mal byť považovaný za autoritatívny zdroj. Pre kritické informácie sa odporúča profesionálny ľudský preklad. Nenesieme zodpovednosť za akékoľvek nedorozumenia alebo nesprávne interpretácie vyplývajúce z použitia tohto prekladu.
